In [12]:
%load_ext autoreload
%autoreload 2
import wallaroo

import os, json
os.environ['DEPLOYMENT_CONFIG'] = json.dumps({"cpus": 0.5, "replica_count": 1, "memory": "500Mi"})

auth_type = "sso"
client = wallaroo.Client(auth_type=auth_type, request_timeout=90, interactive=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Please log into the following URL in a web browser:

	https://secret-peanut-8766.keycloak.wallaroo.dev/auth/realms/master/device?user_code=SDLG-FDFZ

Login successful!


In [13]:
for p in client.list_pipelines(): p.undeploy()

Waiting for undeployment - this will take up to 90s ..................................... ok
Waiting for undeployment - this will take up to 90s ..................................... ok


In [14]:
import pyarrow as pa
input_schema = pa.schema([
    pa.field('dense_2', pa.list_(pa.float64()))
])
output_schema = pa.schema([
    pa.field('output', pa.list_(pa.float64()))
])

In [15]:
from wallaroo.framework import Framework
step = client.upload_model("python-step", 
                           "./step.py", 
                           framework=Framework.PYTHON).configure('python', 
                                                                 input_schema=input_schema, 
                                                                 output_schema=output_schema)


In [16]:
# Deploy a model/pipeline

pipeline_name = 'py-shape3'
model_name = 'py-shape-onnx'

onnx_file_model_name = "house_price_keras.onnx"

fraud = client.upload_model(model_name, onnx_file_model_name, framework=Framework.ONNX).configure('onnx')
pipeline = client.build_pipeline(pipeline_name)
pipeline = pipeline.add_model_step(fraud)

pipeline = pipeline.add_model_step(step)
deployment = pipeline.deploy()

Waiting for deployment - this will take up to 90s ............ ok


In [17]:
import pandas as pd

data = pd.DataFrame.from_dict({"tensor": [[0.6878518042239091,
 0.17607340208535074,
 -0.8695140830357148,
 0.34638762962802144,
 -0.0916270832672289,
 -0.022063226781124278,
 -0.13969884765926363,
 1.002792335666138,
 -0.3067449033633758,
 0.9272000630461978,
 0.28326687982544635,
 0.35935375728372815,
 -0.682562654045523,
 0.532642794275658,
 -0.22705189652659302,
 0.5743846356405602,
 -0.18805086358065454]]})

In [18]:
deployment.infer(data)

,time,in.tensor,out.output,check_failures
0,2023-06-22 17:19:21.158,"[0.6878518042, 0.1760734021, -0.869514083, 0.3...",[12.886651039123535],0


In [19]:
deployment.logs()

,time,in.tensor,out.output,check_failures
0,2023-06-20 20:23:28.395,"[0.6878518042, 0.1760734021, -0.869514083, 0.3...",[12.886651039123535],0
1,2023-06-22 17:19:21.158,"[0.6878518042, 0.1760734021, -0.869514083, 0.3...",[12.886651039123535],0


In [20]:
deployment.undeploy()

Waiting for undeployment - this will take up to 90s .................................... ok


name,py-shape3
created,2023-06-20 20:22:22.407777+00:00
last_updated,2023-06-22 17:19:07.250675+00:00
deployed,False
tags,
versions,"934f01bc-a2ed-4545-b50c-e260bf4d3841, 4dce6d87-2a1f-417e-b872-989a20bf0016, 02c12611-95d3-4c55-86d4-626b49b47dd7, a042827e-fb35-47e9-a18f-67c0d3864237"
steps,py-shape-onnx
